La turbina se encuentra ubicada en Yalova, Turquía. Los datos consisten en mediciones con una frecuencia de 10 min durante el año 2018 de las siguientes variables: 

- LV ActivePower (kW): La potencia instantánea real generada
- Wind Speed (m/s): velocidad del viento medida en el buje de la turbina.
- Theoretical_Power_Curve (KW): Potencia instantánea generada según la curva de rendimiento del fabricante.
- Wind Direction (°): Dirección del viento a la altura del buje de la turbina.

En la siguiente libreta se realizará un análisis descriptivo del rendimiento, exponiéndose: Curva de potencia del año, producciones por mes y pérdidas por mes.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.simplefilter('ignore')


Se cargan los datos y se comprueba la existencia de valores faltantes. Afortundamente, no existen datos nulos. Se realizarán los análisis bajo el supuesto que el sistema de medición funcionó correctamente.

In [ ]:
dataset = pd.read_csv('../input/wind-turbine-scada-dataset/T1.csv', 
                      parse_dates = ['Date/Time'], 
                      date_parser = lambda x: pd.to_datetime(x, format = '%d %m %Y %H:%M'), 
                      index_col = 'Date/Time')


dataset.info()

# transformar a frecuencia horaria y agregar mes

dataset_hora = dataset.resample('H').mean().fillna(0)

dataset_hora['mes'] = dataset_hora.index.strftime('%B')

dataset_hora['mes'] = pd.Categorical(dataset_hora['mes'], 
               categories = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'],
              ordered = True)

dataset_hora.info()

### Gráficas de distribución de las variables

Con estas gráficas se busca establecer una idea general de la distribución de las variables. Se evidencia que:

- Potencial Real y Potencia Teorica, presentan una distribución similar con dos picos, potencia 0 y potencia máxima.

- La velocidad presenta una distribución sesgada hacia la izquierda con dos ligeros picos.

- Respecto a la dirección del viento, se observan dos sentidos predominantes: aproximadamente 90º y 200º


In [ ]:
fig, ax = plt.subplots(2,2, figsize = (15,7.5))
sns.distplot(dataset['LV ActivePower (kW)'], ax = ax[0,0])
sns.distplot(dataset['Wind Speed (m/s)'], ax = ax[0,1])
sns.distplot(dataset['Theoretical_Power_Curve (KWh)'], ax = ax[1,0])
sns.distplot(dataset['Wind Direction (°)'], ax = ax[1,1])
plt.tight_layout()
plt.show()

Los analisis posteriores van a tomar como base la siguente ecuación que describe La potencia generada por turbina eólica: 

\begin{equation}
E = \frac{\rho Av^3}{2}
\end{equation}

Donde:
- $v$, velocidad del viento
- $\rho$, densidad del aire
- $A$, area descrita por los álabes de la turbina

De la ecuación, $A$ es la única variable fija. En operación, la potencia será función de la velocidad y de la densidad del aire, que varían a lo largo del año. De estas, la velocidad es la variable más significativa en la producción de energía; de hecho, la curva característica de una turbina es un gráfico de la potencia generada Vs velocidad del viento. En la figura se establecen tres zonas en función de la magnitud de la velocidad: 

- 1.0 - 3.5 m/s, las turbinas no suelen operar en este rango de velocidad debido a que la potencia generada es baja y no es rentable económicamente la operación. El valor mínimo de operación se conoce como $v_{cut_{in}}$ cuyo valor es, típicamente, 3 - 3.5 m/s.


- 3.5 - 14 m/s, la turbina se encuentra en operación y se evidencia la dependencia cúbica respecto a la velocidad del viento. Para esta turbina, A 14.5 m/s, aproximadamente, se alcanza un máximo en la potencia generada.


- 14 - 25 m/s, el máximo en la curva corresponde a la velocidad de diseño de la turbina, una vez alcanzada, se sigue generando la misma potencia aunque la velocidad aumente. El incremento en la velocidad conlleva un aumento en las cargas mecánicas que debe resistir la estructura por lo que llegada a una cierta magnitud se realiza una parada en la operación por seguridad. Este valor se conoce como $v_{cut_{out}}$ y tiene un valor típico de 25 m/s.


In [ ]:
gb_velocidad = dataset.groupby(pd.qcut(dataset['Wind Speed (m/s)'], q = 25)).mean()

zona_0 = gb_velocidad['Wind Speed (m/s)'] < 4
zona_1 = (gb_velocidad['Wind Speed (m/s)'] >= 3.5) & (gb_velocidad['Wind Speed (m/s)'] <= 15)
zona_2 = gb_velocidad['Wind Speed (m/s)'] >= 14.5

# dataset['Benchmark'] = (dataset['LV ActivePower (kW)'].div(dataset['Theoretical_Power_Curve (KWh)'])).replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(0)

fig, ax = plt.subplots(1,2, figsize = (15,7.5))

# Power Curve grouped

# fill curve zones
ax[0].fill_between(gb_velocidad[zona_0]['Wind Speed (m/s)'], 0 , gb_velocidad[zona_0]['Theoretical_Power_Curve (KWh)'], color = 'gray')
ax[0].fill_between(gb_velocidad[zona_1]['Wind Speed (m/s)'], 0 , gb_velocidad[zona_1]['Theoretical_Power_Curve (KWh)'], color = 'lightgray')
ax[0].fill_between(gb_velocidad[zona_2]['Wind Speed (m/s)'], 0 , gb_velocidad[zona_2]['Theoretical_Power_Curve (KWh)'], color = '#F2F3F4')

#plot 
sns.lineplot(x = 'Wind Speed (m/s)', y = 'LV ActivePower (kW)', data = gb_velocidad, label = 'Real Power (kW)', marker = 'o', ax = ax[0])
sns.lineplot(x = 'Wind Speed (m/s)', y = 'Theoretical_Power_Curve (KWh)', data = gb_velocidad, label = 'Theoretical Power (kW)', marker = 'o', ax = ax[0])
ax[0].legend()

# Power Curve ungrouped
sns.scatterplot(x = 'Wind Speed (m/s)', y = 'LV ActivePower (kW)', data = dataset, label = 'Real Power (kW)', ax = ax[1])
sns.scatterplot(x = 'Wind Speed (m/s)', y = 'Theoretical_Power_Curve (KWh)', data = dataset, label = 'Theoretical Power (kW)', ax = ax[1])
ax[1].legend()


plt.show()

En la siguiente gráfica se muestra la potencia generada por número de horas. Esto permite establecer un contexto sobre la productividad de la turbina. Así, vemos que en un 25% del tiempo, la turbina fue capaz de generar un valor igual o superior a 2360 kW; 760 kW, un 50% del tiempo y 34 kW, un 75% del tiempo. De la gráfica es claro como la turbina estuvo inoperante durante aproximadamente el 25% del tiempo en el año.

In [ ]:
prod_acumulada = dataset_hora['LV ActivePower (kW)'].sort_values(ascending = False)
prod_acumulada = prod_acumulada.reset_index(drop = True)

plt.figure(figsize= (15,7.5))

sns.lineplot(data = prod_acumulada, linewidth = '5')

plt.fill_between( list(range(0, int(8760*0.25))) ,0, prod_acumulada[:int(8760*0.25)], color = 'gray')
plt.fill_between( list(range(int(8760*0.25), int(8760*0.5))), 0, prod_acumulada[int(8760*0.25):int(8760*0.5)], color = 'lightgray')
plt.fill_between( list(range(int(8760*0.5), int(8760*0.75))), 0, prod_acumulada[int(8760*0.5):int(8760*0.75)], color = '#F2F3F4')

plt.title('Potencia generada Vs Número de horas')
plt.ylabel('Potencia [kW]')
plt.xlabel('Horas en el año')
plt.show()

De acuerdo a la ecuación a, se espera encontrar grandes diferencias en la producción de energía a lo largo del año, especialmente, en paises que experimentan las diferentes estaciones. 

En primer lugar, se muestra un análisis por mes para visualizar los efectos de la estacionalidad en la producción de energía. De acuerdo a la gráfica, los meses más fríos (otoño-invierno, hemisferio norte) presentan una producción más alta de energía respecto a los meses más calido, con excepción de Agosto. 

Otro aspecto a resaltar es el factor de uso de la turbina, calculada como: 

\begin{equation}
Factor de uso = \frac{Horas_{operadas}}{Horas_{totales}}
\end{equation}

Se observa lo siguiente:

- Agosto, el mes con mayor disponiblidad
- Enero - Diciembre, meses de invierno, tuvieron la menor disponiblidad
- Está gráfica concuerda con la anterior, la turbina se mantiene en operación al menos un 75% del tiempo en promedio.

In [ ]:
# Agregar mes

dataset['mes'] = dataset.index.strftime('%B')

dataset['mes'] = pd.Categorical(dataset['mes'], 
               categories = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'],
              ordered = True)

# Datos agregados por mes: Produccion de energía por mes y factor de uso 

dataset_mes = dataset_hora.groupby('mes')

agg_mes = dataset_mes.agg( ActivePower_total_MW = ('LV ActivePower (kW)', 'sum'),
                TheoreticalPower_Total_MW = ('Theoretical_Power_Curve (KWh)', 'sum'),
                Velocidad_promedio = ('Wind Speed (m/s)', 'mean'),
                Direccion_promedio = ('Wind Direction (°)', 'mean')
               )
horas_uso = dataset_hora[dataset_hora['LV ActivePower (kW)'] > 0].groupby('mes')['LV ActivePower (kW)'].count()

# horas_uso = dataset[dataset['LV ActivePower (kW)'] > 0].groupby('mes')['LV ActivePower (kW)'].count()

total_horas = dataset_mes['LV ActivePower (kW)'].count()

factor_uso = pd.DataFrame(horas_uso/total_horas).rename(columns = {'LV ActivePower (kW)': 'Factor_uso'})

agg_mes = agg_mes.merge(factor_uso, left_index = True, right_index = True)

agg_mes.sort_index(inplace = True)

agg_mes['ActivePower_total_MW'] = agg_mes['ActivePower_total_MW']/1000 

agg_mes['TheoreticalPower_Total_MW'] = agg_mes['TheoreticalPower_Total_MW']/1000

agg_mes['Perdidas'] = agg_mes['TheoreticalPower_Total_MW'] - agg_mes['ActivePower_total_MW']

# Producción por mes
fig, ax = plt.subplots(2,1, figsize = (15,7.5))

# grafico superior

#top
sns.barplot(x = agg_mes.index , y =  'TheoreticalPower_Total_MW', data = agg_mes, color = 'red', ax = ax[0], label = 'perdidas')
#bottom
sns.barplot(x = agg_mes.index , y =  'ActivePower_total_MW', data = agg_mes, color = 'blue', ax = ax[0], label = 'Energia producida')
ax[0].legend()
ax[0].set_ylabel('Energia Producida_MWh')

# grafico inferior
sns.barplot(x = agg_mes.index, y = 'Factor_uso', data = agg_mes, ax=ax[1], color = 'gray', label = 'Porcentaje de operación')
ax[1].legend()
plt.show()



La gran diferencia en la producción por mes, se debe principalmente en las diferencias en las distribuciones de la velocidad del viento por mes, como se observa en la siguiente gráfica. Aunque, las mayores velocidades se dieron en Marzo, en agosto las velocidades fueron menos dispersas y con valores por encima del promedio anual. En los meses de Abril a Julio, la producción mínima se explica en la baja velocidad del viento.

In [ ]:
# velocidad promedio por mes
plt.figure(figsize = (15,7.5))
sns.boxplot(x = 'mes', y = 'Wind Speed (m/s)', data = dataset_hora, color = '#F2F3F4')
velocidad_promedio = dataset_hora['Wind Speed (m/s)'].mean()
sns.lineplot(x = list(range(0,12)), y = velocidad_promedio, color = 'black', label = f'Velocidad promedio {round(velocidad_promedio,1)} m/s')
plt.legend()
plt.show()

A continuación, se realiza un análisis sobre las diferencias entre la producción real y esperada, de ahora en adelante, pérdidas, las cuales se pueden agrupar por: 
- Horas de operaciones en las que la turbina no operó pero la velocidad era adecuada. Para identificar estos datos se utilizan las velocidades de $v_{cut_{in}}$ y $v_{cut_{out}}$ definidas anteriormente.

- Ineficiencias fluidodinámicas, mecánicas y eléctricas en el sistema álabe-eje-generador-transmisión de la turbina de viento. 

Respecto a la primera, se observa como en los meses de invierno, las pérdidas de energía aumentan considerablemente. En invierno, los álabes pueden sufrir de acumulación de hielo que aumentan las cargas en el rotor además de alterar la fluidodinámica; es un riesgo que se debe controlar durante estos meses. Un aspecto a favor de las bajas temperaturas es que el aire es mas denso. lo que se traduce en un mayor potencial de producción de energía que se debe aprovechar. 

In [ ]:
# Datos fuera de servicio

potencias = []
for i in range(0,110,10):
    
    umbral_potencia = i

    filtros = (dataset_hora['Wind Speed (m/s)'] <= 25) & (dataset_hora['Wind Speed (m/s)'] >= 3.5) & (dataset_hora['LV ActivePower (kW)'] <= umbral_potencia)

    fuera_servicio = dataset_hora[filtros]
    
    potencias.append(fuera_servicio['Theoretical_Power_Curve (KWh)'].sum())

# sns.lineplot(x = range(0,110,10), y = potencias)

# filtrando por fuera de servicio

filtros = (dataset_hora['Wind Speed (m/s)'] <= 25) & (dataset_hora['Wind Speed (m/s)'] >= 3.5) & (dataset_hora['LV ActivePower (kW)'] <= 0)

fuera_servicio = dataset_hora[filtros]

fuera_servicio['Delta_Prod'] = fuera_servicio['Theoretical_Power_Curve (KWh)'] - fuera_servicio['LV ActivePower (kW)']
    
fuera_servicio_gb = fuera_servicio.loc[:,['mes','Theoretical_Power_Curve (KWh)','Delta_Prod']].groupby('mes').sum()

fuera_servicio_gb['Theoretical_Power_Curve (KWh)'] = fuera_servicio_gb['Theoretical_Power_Curve (KWh)']/1000

fuera_servicio_gb['Delta_Prod'] = fuera_servicio_gb['Delta_Prod']/1000

# perdidas por mes

plt.figure(figsize =(15,7.5))
sns.barplot(x = fuera_servicio_gb.index, y = 'Theoretical_Power_Curve (KWh)', data = fuera_servicio_gb, color = 'gray')
plt.ylabel('Potencia perdida [MW]')
plt.title('Pérdidas por fuera de servicio')
plt.show()

Respecto a las pérdidas en operación, destacan tres meses: Agosto, Enero y Diciembre. Los dos últimos se explica, en parte, por lo mencionado anteriormente respecto al problema que enfrentan las turbinas de viento en los meses de invierno. La fluidodinámica se empeora con la acumulación de hielo en los alabes. Sorprende el mes de agosto, estación de verano, donde las pérdidas fueron máximos durante la operación.

In [ ]:
# perdidas de energia turbina en operación
filtros = (dataset_hora['Wind Speed (m/s)'] <= 25) & (dataset_hora['Wind Speed (m/s)'] >= 3.5) & (dataset_hora['LV ActivePower (kW)'] > 0) 
turbina_op = dataset_hora[filtros]

turbina_op['Delta_prod'] = turbina_op['Theoretical_Power_Curve (KWh)'] - turbina_op['LV ActivePower (kW)']

# sns.lineplot(x = turbina_op.index, y =  'Delta_prod', data = turbina_op)
# plt.show()

turbina_op_gb = turbina_op.loc[:,['mes','Delta_prod']].groupby('mes').sum()

turbina_op_gb['Delta_prod'] = turbina_op_gb['Delta_prod']/1000

plt.figure(figsize =(15,7.5))
sns.barplot(x = turbina_op_gb.index, y = 'Delta_prod', data = turbina_op_gb, color = 'gray')
plt.title('Potencia perdida en operación [MW]')
plt.ylabel('MW')
plt.show()

# Composicion de pérdidas

perdidas_totales = turbina_op_gb['Delta_prod'] + fuera_servicio_gb['Delta_Prod'] 

plt.figure(figsize =(15,7.5))
plt.style.use('seaborn-white')
plt.stackplot(perdidas_totales.index, [turbina_op_gb['Delta_prod'],fuera_servicio_gb['Delta_Prod']], labels = ['Pérdidas en operación', 'Pérdidas por fuera de servicio'])
plt.title('Diferencias entre la potencia teórica y real por mes')
plt.ylabel('MW')
plt.legend()
plt.show()

Las altas pérdidas en Agosto pueden deberse a qué también es el mes con mayor producción. Otra forma de analizar las pérdidas por mes durante la operación es ver la distribución de la razón entre las pérdidas y la producción esperada. De la gráfica, se observa que la razón de las pérdidas en Agosto es comparable con los otros meses, una media menor al 20% de desviación. Por otro lado, sobresalen los meses de Abril y Junio, que presentan la mayor media en la desviación, 20%.

In [ ]:
def ratio_perd(mes): 
    
    turbina_op_mes = turbina_op[turbina_op['mes'] == mes]
    sns.boxplot(turbina_op_mes['Delta_prod']/turbina_op_mes['LV ActivePower (kW)'])
    plt.xlim(0,1)
    plt.show()
    
fig, ax = plt.subplots(4,3, sharex = True, figsize = (15, 7.5))

subplots = [item for sublist in ax for item in sublist]

meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

fig.suptitle('Razón de pérdidas/Potencia esperada por mes')
for mes in meses: 
    axis = subplots.pop(0)
    turbina_op_mes = turbina_op[turbina_op['mes'] == mes]
    sns.boxplot(np.abs(turbina_op_mes['Delta_prod'])/turbina_op_mes['Theoretical_Power_Curve (KWh)'], ax  = axis)
    axis.set_title(mes)
    axis.set_xlim(0,1)
#     axis.set_ylim(0,10)
    axis.grid(linewidth=0.25)
    axis.spines['left'].set_visible(False)
    axis.spines['top'].set_visible(False)
    axis.spines['right'].set_visible(False)
    

Del cálculo anterior, también se puede establecer un criterio de filtrado para identificar aquellos días donde se presentaron altas diferencias en las potencias produciadas vs esperadas. De los datos resultantes, se puede seguir investigando para establecer qué sucedió en esos momentos e identificar factores que impactan en la producción de energía. Inicialmente, se establece un valor de 0.3 para filtrar los datos y realizar una investigación a fondo para determinar las causas. Los meses de enero, abril, julio y diciembre sobresalen por la mayor cantidad de datos a analizar.

In [ ]:
shape = []
ratio_list = list(np.arange(0,1.1,0.1))
for i in ratio_list:
    ratio_lim  = np.abs(turbina_op['Delta_prod'])/turbina_op['Theoretical_Power_Curve (KWh)'] >= i
    shape.append(turbina_op[ratio_lim].shape[0])

fig, ax = plt.subplots(1,2, figsize=(15,7.5))
sns.lineplot(x = ratio_list, y = shape, marker = "o", markersize = 8, ax = ax[0])
ax[0].set_title('Número de datos filtrados for umbral de razón de pérdidas')
ax[0].set_xlabel('Razón de pérdidas')

for x,y in zip(ratio_list, shape): 
    
    label = "{:.0f}".format(y)
    ax[0].annotate(label, 
                (x,y), 
                textcoords = 'offset points', 
                xytext = (5,5),
                ha = 'left')
    
    
ratio_lim  = np.abs(turbina_op['Delta_prod'])/turbina_op['Theoretical_Power_Curve (KWh)'] >= 0.3
sns.barplot(x = turbina_op[ratio_lim].groupby('mes').count()['Delta_prod'], 
            y = turbina_op[ratio_lim].groupby('mes').count().index, 
            ax = ax[1], 
           color = 'lightgrey')
ax[1].set_title('Número de datos por encima del umbral de 30% por mes')
ax[1].set_xlabel('Conteo de operaciones filtradas')
plt.show()